# Exercise 3 - Customers of a Hotel Speak

## Hua Wang
## 2016SU_PREDICT_420-DL_SEC55

In [1]:
#Part 1

import pandas as pd
import json

#read json file into jsondat object as dict
with open('100506.json') as input_file:
    jsondat=json.load(input_file)



In [2]:
#process data in the json dicts 
review_list = [] # initiate an empty list

#loop over each review 
for review in jsondat["Reviews"]:
    review_dict = review.copy() #make a copy of review dict
    for key, value in review.items():  #loop over dict and get key, value
        if key == 'Ratings':     #for Ratings, merge to review_dict, and append to review_list
            review_dict.update(value)
            review_list.append(review_dict)
 
#convert list of dicts to dataframe
review_df = pd.DataFrame.from_records(review_list)
#drop unneeded columns
review_df = review_df.drop(['AuthorLocation', 'Ratings','ReviewID','Title'], axis=1) 

#change ratings into numeric
review_df[['Business service (e.g., internet access)','Check in / front desk','Cleanliness','Location', \
           'Overall','Rooms','Service','Sleep Quality','Value']] \
= review_df[['Business service (e.g., internet access)','Check in / front desk','Cleanliness','Location', \
           'Overall','Rooms','Service','Sleep Quality','Value']].apply(pd.to_numeric)
review_df.head() #show dataframe

        
            
            
            

            


    
    




,Author,"Business service (e.g., internet access)",Check in / front desk,Cleanliness,Content,Date,Location,Overall,Rooms,Service,Sleep Quality,Value
0,luvsroadtrips,NaN,NaN,1,This place is not even suitable for the homele...,"January 3, 2012",5,1,1,1,1,1
1,estelle e,NaN,NaN,4,We stayed in downtown hotel Seattle for two ni...,"December 29, 2011",5,4,3,4,5,3
2,RobertEddy,NaN,NaN,2,"i made reservations and when i showed up, i qu...","December 20, 2011",1,1,1,1,1,1
3,James R,NaN,NaN,1,This hotel is so bad it's a joke. I could bare...,"October 30, 2011",1,1,1,1,1,1
4,Shobha49,NaN,NaN,NaN,My husband and I stayed at this hotel from 16t...,"September 14, 2011",5,1,1,1,NaN,3


In [9]:
# print the mean, min, and max for each ratings
for rating in ['Business service (e.g., internet access)','Check in / front desk','Cleanliness','Location', \
           'Overall','Rooms','Service','Sleep Quality','Value']:
    print ("The mean for " + rating + ' = ' +  str(review_df[rating].mean()) + ', min = ' + str(review_df[rating].min()) \
          + ', max = ' + str(review_df[rating].max())) 


The mean for Business service (e.g., internet access) = 1.0, min = 1.0, max = 1.0
The mean for Check in / front desk = 3.0, min = 1.0, max = 5.0
The mean for Cleanliness = 2.0, min = 1.0, max = 5.0
The mean for Location = 4.0, min = 1.0, max = 5.0
The mean for Overall = 1.6666666666666667, min = 1.0, max = 4.0
The mean for Rooms = 1.5454545454545454, min = 1.0, max = 5.0
The mean for Service = 2.3, min = 1.0, max = 5.0
The mean for Sleep Quality = 2.176470588235294, min = 1.0, max = 5.0
The mean for Value = 2.0, min = 1.0, max = 5.0


In [4]:
comment_df = review_df[['Author', 'Date','Content']]
comment_df = comment_df.set_index('Author') #set author as index
comment_df['Content'] = comment_df['Content'].apply(str) #convert the content column to string

#export dataframe comment_df to text where each reviewer in one row
comment_df.to_csv(r'100506_review_comments.txt', header=None, index=None, sep=' ', mode='a')

#save review_df to pickle
review_df.to_pickle('100506_ratings.pkl')


In [5]:
# Part 2
import warnings
warnings.filterwarnings("ignore") #surpress warnings

dirname = "/Users/huawang/Documents/classes/Northwestern/420/ex3"
files = [] #initialize an empty files list

import os
for file in os.listdir(dirname):
    if file.endswith(".json"): #find all files end with .json
        files.append(file) #append to list
print (files)

['100506.json', '1217974.json', '677703.json']


In [6]:
hotels = [] #initialize an empty list hotels


for file in files:
    with open(file) as input_file:
        jsondat=json.load(input_file)
        hotels.append(jsondat['HotelInfo'])
hotels_df = pd.DataFrame.from_records(hotels) #convert list of dicts into dataframe
hotels_df

,Address,HotelID,HotelURL,ImgURL,Name,Price
0,"<address class=""addressReset""> <span rel=""v:ad...",100506,/ShowUserReviews-g60878-d100506-Reviews-Hotel_...,http://media-cdn.tripadvisor.com/media/Provide...,Hotel Seattle,$96 - $118*
1,NaN,1217974,http://www.tripadvisor.com/ShowUserReviews-g60...,NaN,NaN,Unkonwn
2,NaN,677703,http://www.tripadvisor.com/ShowUserReviews-g15...,NaN,NaN,Unkonwn


In [7]:
#create a function to parse Address to reable address string
from bs4 import BeautifulSoup
def parse_html (x):
    soup = BeautifulSoup(str(x))
    [s.extract() for s in soup(['street-address', 'locality', 'region', 'postal-code'])]
    readable_text = soup.getText()
    return readable_text.strip(' ')

hotels_df['Address'] = hotels_df['Address'].apply(parse_html) #apply function to Address column
hotels_df 

,Address,HotelID,HotelURL,ImgURL,Name,Price
0,"315 Seneca St., Seattle, WA 98101",100506,/ShowUserReviews-g60878-d100506-Reviews-Hotel_...,http://media-cdn.tripadvisor.com/media/Provide...,Hotel Seattle,$96 - $118*
1,nan,1217974,http://www.tripadvisor.com/ShowUserReviews-g60...,NaN,NaN,Unkonwn
2,nan,677703,http://www.tripadvisor.com/ShowUserReviews-g15...,NaN,NaN,Unkonwn


In [8]:
#pickle dataframe
hotels_df.to_pickle('HotelInfo.pkl')